# Combine data from flood, crime and HCAD from Selected ZIP Codes


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime
from scipy.stats import linregress
import math



In [2]:
#Merge all flood csv data files in a dataframe
ZIP=['77002','77005','77006','77019','77025','77027','77030','77054','77098']

flood_all_zips_df=pd.DataFrame()
for z in ZIP:
    file=f'Output/flood_data_{z}.csv'
    df=pd.read_csv(file)
    flood_all_zips_df=flood_all_zips_df.append(df)
for x in range(0,9):
    file=f'Output/flood_data_batch{x}.csv'
    df=pd.read_csv(file)
    flood_all_zips_df=flood_all_zips_df.append(df)
    
del flood_all_zips_df['Unnamed: 0']
flood_all_zips_df=flood_all_zips_df.reset_index(drop=True)
flood_all_zips_df.sample(5)

,Address,Latitude,Longitude,Flood Description,Flood Zone
9049,"1906 PARK ST HOUSTON, Texas 77019",29.74888,-95.40323,AREA OF MINIMAL FLOOD HAZARD,X
19241,"3433 CLEARVIEW CIR HOUSTON, Texas 77025",29.66729,-95.43458,AREA OF MINIMAL FLOOD HAZARD,X
23742,"2400 SWIFT BLVD HOUSTON, Texas 77030",29.71286,-95.41506,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,X
19510,"3738 MAIN POPLAR DR HOUSTON, Texas 77025",29.66256,-95.43873,AREA OF MINIMAL FLOOD HAZARD,X
25001,"1912 HAROLD ST HOUSTON, Texas 77098",29.74137,-95.40753,AREA OF MINIMAL FLOOD HAZARD,X


In [3]:
flood_all_zips_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30981 entries, 0 to 30980
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Address            30981 non-null  object 
 1   Latitude           30981 non-null  float64
 2   Longitude          30981 non-null  float64
 3   Flood Description  25510 non-null  object 
 4   Flood Zone         30981 non-null  object 
dtypes: float64(2), object(3)
memory usage: 1.2+ MB


In [4]:
#Add flood ranking
#3- High Risk
#2 - Medium Risk
#1- Low Risk

flood_all_zips_df['Flood Risk']=np.where(flood_all_zips_df['Flood Description']=='AREA OF MINIMAL FLOOD HAZARD',1," ")
flood_all_zips_df.loc[(flood_all_zips_df['Flood Description']=='0.2 PCT ANNUAL CHANCE FLOOD HAZARD'),'Flood Risk']=2
flood_all_zips_df.loc[(flood_all_zips_df['Flood Zone']=='AE'),'Flood Risk']=3
flood_all_zips_df.reset_index(drop=True)
flood_all_zips_df.sample(5)

,Address,Latitude,Longitude,Flood Description,Flood Zone,Flood Risk
11465,"2912 ELLA LEE LN HOUSTON, Texas 77019",29.74595,-95.41957,AREA OF MINIMAL FLOOD HAZARD,X,1
17638,"3622 DEAL ST HOUSTON, Texas 77025",29.68924,-95.43275,NaN,AE,3
28458,"1625 VASSAR ST, Houston, Texas 77006",29.73020,-95.40048,AREA OF MINIMAL FLOOD HAZARD,X,1
18708,"4022 GRAMERCY ST HOUSTON, Texas 77025",29.70443,-95.44156,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,X,2
9790,"2201 AVALON PL HOUSTON, Texas 77019",29.74660,-95.41366,AREA OF MINIMAL FLOOD HAZARD,X,1


In [5]:
flood_all_zips_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30981 entries, 0 to 30980
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Address            30981 non-null  object 
 1   Latitude           30981 non-null  float64
 2   Longitude          30981 non-null  float64
 3   Flood Description  25510 non-null  object 
 4   Flood Zone         30981 non-null  object 
 5   Flood Risk         30981 non-null  object 
dtypes: float64(2), object(4)
memory usage: 1.4+ MB


In [6]:
#Merge Flood and HCAD DataFrames to get Account Number, Neighborhood code and group and rest of parameters

#Generate a colum that has SITE_ADDR_1
address_split=flood_all_zips_df['Address'].str.rsplit(' ',3).tolist()
address_split=pd.DataFrame(address_split)
flood_all_zips_df['SITE_ADDR_1']= address_split[0]
flood_all_zips_df.sample(5)



,Address,Latitude,Longitude,Flood Description,Flood Zone,Flood Risk,SITE_ADDR_1
12455,"1018 LA RUE ST HOUSTON, Texas 77019",29.75673,-95.39577,AREA OF MINIMAL FLOOD HAZARD,X,1,1018 LA RUE ST
21408,"3506 LAS PALMAS ST HOUSTON, Texas 77027",29.73408,-95.44447,AREA OF MINIMAL FLOOD HAZARD,X,1,3506 LAS PALMAS ST
1382,"2132 RICE BLVD HOUSTON, Texas 77005",29.71905,-95.40857,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,X,2,2132 RICE BLVD
19635,"4047 ABERDEEN WAY HOUSTON, Texas 77025",29.69774,-95.44266,NaN,AE,3,4047 ABERDEEN WAY
28633,"1652 KIPLING ST, Houston, Texas 77006",29.74061,-95.40150,AREA OF MINIMAL FLOOD HAZARD,X,1,"1652 KIPLING ST,"


In [7]:
# open hcad data
HCAD_df=pd.read_csv('Inner_Loop_18_19.csv')
HCAD_df=HCAD_df.drop(["Unnamed: 0"], axis=1)
HCAD_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32194 entries, 0 to 32193
Data columns (total 19 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   ACCOUNT                                     32194 non-null  int64  
 1   SITE_ADDR_1                                 32194 non-null  object 
 2   SITE_ADDR_3                                 32194 non-null  int64  
 3   NEIGHBORHOOD_CODE                           32194 non-null  float64
 4   NEIGHBORHOOD_GROUP                          32194 non-null  float64
 5   TOTAL_BUILDING_AREA                         32194 non-null  float64
 6   TOTAL_LAND_AREA                             32194 non-null  float64
 7   ACREAGE                                     32194 non-null  float64
 8   LAND_VALUE_2019                             32194 non-null  float64
 9   TOTAL_APPRAISED_VALUE_2019                  32194 non-null  float64
 10  TOTAL_MARK

In [8]:
# # get a list of missing addresses to run through the flood data api
# addresses=[]
# haddr=HCAD_df["SITE_ADDR_1"]
# faddr=flood_all_zips_df["SITE_ADDR_1"]

# missing_addresses = np.setdiff1d(haddr,faddr)
# missing_addresses=missing_addresses.tolist()
# missing_addresses
# need_addresses=pd.DataFrame()
# need_addresses["address"]=missing_addresses
# zip_code=[]
# for x in range(len(missing_addresses)):
#     row_df=HCAD_df.loc[HCAD_df["SITE_ADDR_1"]==missing_addresses[x]]
#     row_df
#     zipp=row_df["SITE_ADDR_3"].values
#     zip_code.append(zipp[0])
# zip_code    
# need_addresses["zip_code"]=zip_code
# need_addresses

In [9]:
# # write the list to csv for open in flood api
# need_addresses.to_csv("missing_addresses.csv")

In [11]:
#Merge Flood data to HCAD by SITE_ADDR_1
HCAD_df = HCAD_df.drop_duplicates(subset=['SITE_ADDR_1'])
flood_all_zips_df = flood_all_zips_df.drop_duplicates(subset=['SITE_ADDR_1'])

flood_all_zips_df=pd.DataFrame.merge(flood_all_zips_df,HCAD_df,on='SITE_ADDR_1', how='inner', suffixes=('_flood', '_hcad'))

# flood_all_zips_df.to_csv('Output/flood_all_zips.csv')
flood_all_zips_df.sample(5)

,Address,Latitude,Longitude,Flood Description,Flood Zone,Flood Risk,SITE_ADDR_1,ACCOUNT,SITE_ADDR_3,NEIGHBORHOOD_CODE,...,TOTAL_APPRAISED_VALUE_2019,TOTAL_MARKET_VALUE_2019,LAND_VALUE_2018,TOTAL_APPRAISED_VALUE_2018,TOTAL_MARKET_VALUE_2018,pct_change_land_value_2018_2019,pct_change_total_appraised_value_2018_2019,pct_change_total_market_value_2018_2019,SQ_FT,NEIGHBORHOOD
10730,"2 SHADDER WAY HOUSTON, Texas 77019",29.75931,-95.41453,NaN,AE,3,2 SHADDER WAY,601580000006,77019,8323.05,...,6660060.0,6660060.0,3699108.0,6601120.0,6601120.0,0.000000,0.892879,0.892879,6058,RIVER OAKS NORTH OF KIRBY
19845,"2708 WEST LANE DR D HOUSTON, Texas 77027",29.74009,-95.44966,AREA OF MINIMAL FLOOD HAZARD,X,1,2708 WEST LANE DR D,720780000050,77027,8314.04,...,791950.0,792719.0,246038.0,719955.0,719955.0,36.460628,9.999931,10.106743,3395,WEST LANE PLACE ROW T/H
10188,"2941 CHEVY CHASE DR HOUSTON, Texas 77019",29.74873,-95.42060,AREA OF MINIMAL FLOOD HAZARD,X,1,2941 CHEVY CHASE DR,601500290007,77019,8325.00,...,3649700.0,3649700.0,1575000.0,2885605.0,2885605.0,0.000000,26.479542,26.479542,7054,"RIVER OAKS 4, 5, PT CCE"
12491,"1204 RUTHVEN ST HOUSTON, Texas 77019",29.75533,-95.37979,AREA OF MINIMAL FLOOD HAZARD,X,1,1204 RUTHVEN ST,1246690040003,77019,8318.01,...,249637.0,298000.0,137790.0,226943.0,294858.0,0.000000,9.999868,1.065598,1483,FOURTH WARD DEED RESTR
19829,"2606 WEST LANE DR HOUSTON, Texas 77027",29.74069,-95.44986,AREA OF MINIMAL FLOOD HAZARD,X,1,2606 WEST LANE DR,720780000011,77027,8314.04,...,921847.0,1042082.0,319538.0,838043.0,838043.0,32.674361,9.999964,24.347080,4362,WEST LANE PLACE ROW T/H


In [12]:
flood_all_zips_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26253 entries, 0 to 26252
Data columns (total 25 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Address                                     26253 non-null  object 
 1   Latitude                                    26253 non-null  float64
 2   Longitude                                   26253 non-null  float64
 3   Flood Description                           20837 non-null  object 
 4   Flood Zone                                  26253 non-null  object 
 5   Flood Risk                                  26253 non-null  object 
 6   SITE_ADDR_1                                 26253 non-null  object 
 7   ACCOUNT                                     26253 non-null  int64  
 8   SITE_ADDR_3                                 26253 non-null  int64  
 9   NEIGHBORHOOD_CODE                           26253 non-null  float64
 10  NEIGHBORHO

In [13]:
HCAD_df["SITE_ADDR_3"].count()

29978

In [14]:
#Pull Crime Rate per ZIP Code and Rank
crime_df=pd.read_csv('Resources/2019_Houston_Crimes.csv')
crime_zips=crime_df.groupby(['ZIP Code'])
crime_zips=crime_zips['Offense Count'].sum()[ZIP]
crime_zips_df=pd.DataFrame(crime_zips)
crime_zips_df=crime_zips_df.reset_index()
max=crime_zips_df['Offense Count'].max()
min=crime_zips_df['Offense Count'].min()
crime_zips_df['crime_rank']=1-(crime_zips_df['Offense Count']-min)/(max-min)
crime_zips_df['ZIP Code']=pd.to_numeric(crime_zips_df['ZIP Code'], errors='coerce')
crime_zips_df



,ZIP Code,Offense Count,crime_rank
0,77002,4870,0.000000
1,77005,1026,1.000000
2,77006,3759,0.289022
3,77019,2330,0.660770
4,77025,2270,0.676379
5,77027,2014,0.742976
6,77030,1485,0.880593
7,77054,3053,0.472685
8,77098,2051,0.733351


In [15]:
#Add Crime Rate to Flood and HCAD dataframe.
#Export file with all data 
flood_crime_hcad_df=pd.merge(flood_all_zips_df,crime_zips_df,left_on='SITE_ADDR_3', right_on='ZIP Code')
flood_crime_hcad_df=flood_crime_hcad_df.drop(['ZIP Code'],axis=1)
flood_crime_hcad_df.sample(5)



,Address,Latitude,Longitude,Flood Description,Flood Zone,Flood Risk,SITE_ADDR_1,ACCOUNT,SITE_ADDR_3,NEIGHBORHOOD_CODE,...,LAND_VALUE_2018,TOTAL_APPRAISED_VALUE_2018,TOTAL_MARKET_VALUE_2018,pct_change_land_value_2018_2019,pct_change_total_appraised_value_2018_2019,pct_change_total_market_value_2018_2019,SQ_FT,NEIGHBORHOOD,Offense Count,crime_rank
1084,"2706 CASON ST HOUSTON, Texas 77005",29.70816,-95.42049,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,X,2,2706 CASON ST,511170060002,77005,7423.00,...,635250.0,1418900.0,1418900.0,14.285714,2.364226,2.364226,4797,WEST U PEMBERTON,1026,1.000000
4480,"3126 ROBINHOOD ST HOUSTON, Texas 77005",29.72111,-95.42714,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,X,2,3126 ROBINHOOD ST,590450000018,77005,7425.01,...,970200.0,1202000.0,1202000.0,14.285714,-0.166389,-0.166389,2510,"WEST U,MONTICELLO",1026,1.000000
24674,"2014 SUL ROSS ST HOUSTON, Texas 77098",29.73793,-95.40874,AREA OF MINIMAL FLOOD HAZARD,X,1,2014 SUL ROSS ST,561210000019,77098,8316.02,...,496875.0,561046.0,561046.0,6.666667,1.878634,1.878634,1876,CHERRYHURST/OTHERS,2051,0.733351
24313,"1909 BRANARD ST H HOUSTON, Texas 77098",29.73666,-95.40725,AREA OF MINIMAL FLOOD HAZARD,X,1,1909 BRANARD ST H,382200000003,77098,8316.05,...,134840.0,347579.0,347579.0,0.000000,0.000000,0.000000,2196,HYDE PARK MAIN 6-ROW T/H,2051,0.733351
18957,"3013 CLEARVIEW CIR HOUSTON, Texas 77025",29.66884,-95.43415,AREA OF MINIMAL FLOOD HAZARD,X,1,3013 CLEARVIEW CIR,1269180010006,77025,7454.06,...,68801.0,266302.0,266302.0,0.000000,-4.216266,-4.216266,1734,PARK AT CLEARVIEW & AMENDING NO 1,2270,0.676379


In [16]:
flood_crime_hcad_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26253 entries, 0 to 26252
Data columns (total 27 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Address                                     26253 non-null  object 
 1   Latitude                                    26253 non-null  float64
 2   Longitude                                   26253 non-null  float64
 3   Flood Description                           20837 non-null  object 
 4   Flood Zone                                  26253 non-null  object 
 5   Flood Risk                                  26253 non-null  object 
 6   SITE_ADDR_1                                 26253 non-null  object 
 7   ACCOUNT                                     26253 non-null  int64  
 8   SITE_ADDR_3                                 26253 non-null  int64  
 9   NEIGHBORHOOD_CODE                           26253 non-null  float64
 10  NEIGHBORHO

In [17]:
flood_crime_hcad_df.to_csv('Output/flood_crime_hcad.csv')

In [1]:
flood_crime_hcad_df["ACCOUNT"].value_counts()

NameError: name 'flood_crime_hcad_df' is not defined